## Alineamiento y Cálculo de nuevos Centroides

__Librerías y definición de funciones.__

In [3]:
import os
import random
import numpy as np
import utils.bundleTools as bt
import utils.bundleTools3 as bt3
import utils.visualizationTools as vt
from dipy.segment.metric import mdf
from dipy.segment.metric import dist
from dipy.segment.metric import EuclideanMetric
from time import time

def align_cluster(cluster, ref):
    
    aligned_cluster = []
    
    for fiber in cluster:
        d = dist(EuclideanMetric(), fiber, ref)/21
        f = mdf(fiber, ref)
        
        if f < d:
            aligned_cluster.append(fiber[::-1])
            
        else:
            aligned_cluster.append(fiber)
            
    return aligned_cluster

def calculate_centroid(cluster):
    return np.mean(cluster, axis = 0)

byte_order = "DCBA"

def random_palette(n):
    return [(random.randrange(0, 255), random.randrange(0, 255), random.randrange(0, 255)) for i in range(n)]

def write_hie(path,names,colors):
    f = open(path,"w")
    f.write("# tree 1.0\n\n*BEGIN TREE hierarchy\ngraph_syntax RoiArg\n\n*BEGIN TREE fold_name\nname ALL\n\n")
    for i,n in enumerate(names):
        f.write("*BEGIN TREE fold_name\n")
        f.write("name "+n+"\n")
        f.write("color "+str(colors[i][0])+" "+str(colors[i][1])+" "+str(colors[i][2])+"\n\n")
        f.write("*END\n\n")
    f.write("*END\n\n*END\n\n*END\n\n")
    f.close()


def write_header(path,bnames,intervals,dim,nfibers):
    f = open(path,"w")
    f.write("attributes = {\n    'binary' : 1,\n    'bundles' : ")
    f.write(str([val for pair in zip(bnames, intervals) for val in pair])+",\n")
    f.write("    'byte_order' : "+'\''+byte_order+'\',\n')
    f.write("    'curves_count' : "+str(nfibers)+",\n")
    f.write("    'data_file_name' : "+"\'*"+os.path.splitext(path+"data")[1]+"\',\n")
    f.write("    'format' : 'bundles_1.0',\n")
    f.write("    'space_dimension' : "+str(dim)+"\n  }")
    f.close()

def write_data(data_path,bundles,dim):
    f = open(data_path,"wb")
    for b in bundles:
        for fiber in b:
            f.write(np.array([len(fiber)],dtype=np.int32).tostring())
            f.write(fiber.ravel().tostring())
    f.close()

def write_bundles(path,bundles,bnames=None,colors=None):
    if bnames == None:
        bnames = [''+str(i)+'' for i in range(len(bundles))]
    bnames = [''+n.strip()+'' for n in bnames]
    data_path = path+"data"
    dim = len(bundles[0][0][0])
    nfibers = sum(len(b) for b in bundles)
    intervals = [0]
    for i in range(1,len(bundles),1):
        intervals.append(intervals[i-1]+len(bundles[i-1])) 

    write_header(path,bnames,intervals,dim,nfibers)
    write_data(data_path,bundles,dim)
    if colors == None:
        colors = random_palette(len(bnames))
    write_hie(os.path.splitext(path)[0]+".hie",bnames,colors)

def getBundleNames2( bundlefile ):
    #get center names from bundle file
    ns = dict()
    exec(compile(open( bundlefile ).read(), bundlefile, 'exec'), ns)
    bunlist = ns[ 'attributes' ][ 'bundles' ]
    centers_num = len( bunlist ) // 2
    labels = []
    for i in range( centers_num ):
        ind = i * 2
        labels.append( bunlist[ ind ] )
    return labels


__Definición de nombres y rutas.__

In [4]:
#Parámetros de HDBSCAN.
ms = 1
mcs = 65

#Parámetros de QuickBundles.
theta = 20

#Seleccionar cual clustering utilizar.
c_types = ['QB', 'HDB']
select = 0
c_type = c_types[select]

#Se cargan los centroides de referencia.
if c_type == 'QB':
    data_path = 'data/79subjects/inter-clustered_QB_' + str(theta) + '2/'
    ref_centroids = bt3.read_bundle_severalbundles(data_path + 'QB_' + str(theta) + '_centroids.bundles')
    
elif c_type == 'HDB':
    data_path = 'data/79subjects/inter-clustered_HDB_' + str(ms) + '_' + str(mcs) + '2/'
    ref_centroids = bt3.read_bundle_severalbundles(data_path + 'HDB_' + str(ms) + '_' + str(mcs) + '_centroids.bundles')


data/79subjects/inter-clustered_QB_202/QB_20_centroids.bundlesdata


En lo anterior, centroides de referencia (ref_centroids) se refiere a los centroides de los clusters ínter-sujeto obtenidos con HDBSCAN/QuickBundles. En el segundo caso, estos ya fueron calculados por QB.

__Alineamiento y fusión de clusters__

In [6]:
t0 = time();

#Para cada sujeto...
for i in range(1,80):
    
    #Listas con nuevos clusters y centroides
    sub_clusters = []
    sub_centroids = []
    
    if i < 10:
        sub = '00' + str(i)
    else:
        sub = '0' + str(i)
    
    #Lectura de clusters de cada sujeto
    clusters_obj = bt3.read_bundle_severalbundles(data_path + sub + '/clusters.bundles')
    clusters = clusters_obj[0]
    
    #Nombres de los clusters (con respecto a los ínter-sujeto)
    names = clusters_obj[1]
    
    #Lectura de centroides
    centroids = bt3.read_bundle_severalbundles(data_path + sub + '/centroids.bundles')[0]

    #Para cada cluster...
    for idx, cluster in enumerate(clusters):
        
        #Se obtiene el nombre del cluster en cuestión
        cluster_name = names[idx]
        
        #Obtenemos el índice del cluster en la lista de nombres de centroides de referencia...
        ref_idx = ref_centroids[1].index(cluster_name)
        
        #...y con el índice obtenemos el centroide correspondiente (a nivel ínter-sujeto)
        ref = ref_centroids[0][ref_idx][0]
        
        #Tomamos todas las fibras del cluster y las alineamos con respecto al centroide ínter-sujeto
        cluster_a = align_cluster(cluster, ref)
        
        #Teniendo todas las fibras alineadas, calculamos el nuevo centroide del cluster ínter-sujeto a nivel intra-sujeto (promedio geométrico del aporte del sujeto al cluster ínter-sujeto)
        centroid = calculate_centroid(cluster_a)
        
        sub_clusters.append(cluster_a)
        sub_centroids.append([centroid])
        
    if not os.path.exists(data_path + sub):
        os.makedirs(data_path + sub)
    
    #Se guardan los clusters y centroides finales. Ahora Final_Centroids sí contiene sólo un centroide por cluster.
    bt3.write_bundle_severalbundles(data_path + sub + '/Final_Clusters.bundles', sub_clusters, names)
    bt3.write_bundle_severalbundles(data_path + sub + '/Final_Centroids.bundles', sub_centroids, names)
    
print('That took ' + str(time() - t0) + ' [s]')

data/79subjects/inter-clustered_QB_202/001/clusters.bundlesdata
data/79subjects/inter-clustered_QB_202/001/centroids.bundlesdata
data/79subjects/inter-clustered_QB_202/002/clusters.bundlesdata
data/79subjects/inter-clustered_QB_202/002/centroids.bundlesdata
data/79subjects/inter-clustered_QB_202/003/clusters.bundlesdata
data/79subjects/inter-clustered_QB_202/003/centroids.bundlesdata
data/79subjects/inter-clustered_QB_202/004/clusters.bundlesdata
data/79subjects/inter-clustered_QB_202/004/centroids.bundlesdata
data/79subjects/inter-clustered_QB_202/005/clusters.bundlesdata
data/79subjects/inter-clustered_QB_202/005/centroids.bundlesdata
data/79subjects/inter-clustered_QB_202/006/clusters.bundlesdata
data/79subjects/inter-clustered_QB_202/006/centroids.bundlesdata
data/79subjects/inter-clustered_QB_202/007/clusters.bundlesdata
data/79subjects/inter-clustered_QB_202/007/centroids.bundlesdata
data/79subjects/inter-clustered_QB_202/008/clusters.bundlesdata
data/79subjects/inter-clustered_Q

data/79subjects/inter-clustered_QB_202/065/clusters.bundlesdata
data/79subjects/inter-clustered_QB_202/065/centroids.bundlesdata
data/79subjects/inter-clustered_QB_202/066/clusters.bundlesdata
data/79subjects/inter-clustered_QB_202/066/centroids.bundlesdata
data/79subjects/inter-clustered_QB_202/067/clusters.bundlesdata
data/79subjects/inter-clustered_QB_202/067/centroids.bundlesdata
data/79subjects/inter-clustered_QB_202/068/clusters.bundlesdata
data/79subjects/inter-clustered_QB_202/068/centroids.bundlesdata
data/79subjects/inter-clustered_QB_202/069/clusters.bundlesdata
data/79subjects/inter-clustered_QB_202/069/centroids.bundlesdata
data/79subjects/inter-clustered_QB_202/070/clusters.bundlesdata
data/79subjects/inter-clustered_QB_202/070/centroids.bundlesdata
data/79subjects/inter-clustered_QB_202/071/clusters.bundlesdata
data/79subjects/inter-clustered_QB_202/071/centroids.bundlesdata
data/79subjects/inter-clustered_QB_202/072/clusters.bundlesdata
data/79subjects/inter-clustered_Q